In [1]:
import os
main_path='UTKface_Aligned_cropped/crop_part1/'
d=os.listdir(main_path)
count=0
dict={}
for i in range(1,120):
    dict[i]=[]


for i,dirs in enumerate(sorted(d)):
    age=dirs.split("_")[0]
    dict[int(age)].append(dirs)


In [2]:
age_grp=['1-10','11-20','21-30','31-40','41-50','51-60']
main_dict={}
for key in age_grp:
    main_dict[key]=[]
for key in age_grp:
    k=key.split("-")
    for i in range(int(k[0]),int(k[1])+1):
        count=0
        main_dict[key].append(dict[i][0:25])
    


In [3]:
for k in main_dict.keys():
    arr=[]
    for el in main_dict[k]:
        for item in el:
            arr.append(item)
    main_dict[k]=arr

In [4]:
import cv2 as cv
for key in main_dict.keys():
    for pic in main_dict[key]:
        img=cv.imread(main_path+pic)
        
        

In [5]:
num_dict={}
count=0
for key in main_dict.keys():
    count=0
    for pic in main_dict[key]:
        count+=1
    num_dict[key]=count

In [6]:
num_dict

{'1-10': 250,
 '11-20': 250,
 '21-30': 250,
 '31-40': 250,
 '41-50': 250,
 '51-60': 250}

In [7]:
import keras

/Users/avisheksarkar/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
import os
import numpy as np
import cv2 as cv
import dlib
%matplotlib inline
import matplotlib.pyplot as plt
face_detector = dlib.get_frontal_face_detector()

predictor_model="shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_model)
facerec=dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
d=os.listdir(main_path)
main_path=os.getcwd()+'/UTKface_Aligned_cropped/crop_part1/'
main_path

'/Users/avisheksarkar/Desktop/final_year_project/UTKface_Aligned_cropped/crop_part1/'

In [9]:
img_array=[]
img_labels=[]
import cv2 as cv
for i,key in enumerate(main_dict.keys()):
    print(key)
    label=[0,0,0,0,0,0]
    for pic in main_dict[key]:
        
        img=cv.imread(main_path+pic)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        face_rect= face_detector(gray, 1)
        for k,d in enumerate(face_rect):
            shape=predictor(gray,d)
            face_descriptor = facerec.compute_face_descriptor(img,shape )
                
        label[i]=1
        img_array.append(face_descriptor)
        img_labels.append(label)
img_dataset=np.array(img_array)
img_labell=np.array(img_labels)
print(img_dataset.shape)
print(img_labell.shape)                

1-10
11-20
21-30
31-40
41-50
51-60
(1500, 128)
(1500, 6)


In [10]:
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(img_dataset,
	img_labell, test_size=0.25, random_state=42)

In [11]:
img_labell[-1]

array([0, 0, 0, 0, 0, 1])

In [37]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
model = Sequential()
model.add(Dense(1024, input_dim=128, activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(6, activation='softmax'))
#sgd = optimizers.SGD(lr=0.063, decay=0.9)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
model.fit(img_dataset, img_labell, epochs=15, shuffle=True,batch_size=20,validation_split=0.15)

Train on 1275 samples, validate on 225 samples
Epoch 1/15
1275/1275 [==============================] - 1s - loss: 0.2730 - acc: 0.8685 - val_loss: 0.8075 - val_acc: 0.7496
Epoch 2/15
1275/1275 [==============================] - 1s - loss: 0.2702 - acc: 0.8684 - val_loss: 0.6852 - val_acc: 0.7807
Epoch 3/15
1275/1275 [==============================] - 1s - loss: 0.2688 - acc: 0.8680 - val_loss: 0.7761 - val_acc: 0.6770
Epoch 4/15
1275/1275 [==============================] - 1s - loss: 0.2627 - acc: 0.8733 - val_loss: 0.7475 - val_acc: 0.6926
Epoch 5/15
1275/1275 [==============================] - 1s - loss: 0.2617 - acc: 0.8724 - val_loss: 1.0263 - val_acc: 0.6763
Epoch 6/15
1275/1275 [==============================] - 1s - loss: 0.2656 - acc: 0.8733 - val_loss: 0.8051 - val_acc: 0.6956
Epoch 7/15
1275/1275 [==============================] - 1s - loss: 0.2637 - acc: 0.8705 - val_loss: 0.8042 - val_acc: 0.6741
Epoch 8/15
1275/1275 [==============================] - 1s - loss: 0.2653 - ac

In [45]:
alpha=['baby','teen','young','mid-age','old','very old']
cap = cv.VideoCapture(0)
ret,frame=cap.read()
while ret:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#image=os.getcwd()+'/test_images/srija_test.jpeg'
#image=cv.imread(image)
#gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    detected_faces = face_detector(gray, 1)
    if(detected_faces):
        for i,face_rect in enumerate(detected_faces):
            shape=predictor(gray,face_rect)
            face_descriptor = facerec.compute_face_descriptor(frame,shape)
            face_descriptor=np.array(face_descriptor)
            face_descriptor=face_descriptor.reshape((-1,1*128)).astype(np.float32)
            print(model.predict(face_descriptor))

[[0.00180773 0.12759055 0.6461065  0.17555496 0.04380823 0.00513201]]
[[6.4001841e-05 8.5844705e-03 2.3177977e-01 3.8947758e-01 3.2916719e-01
  4.0926974e-02]]
[[1.2491153e-04 3.2963637e-02 6.1677861e-01 2.6098248e-01 7.4529946e-02
  1.4620399e-02]]
[[5.7044988e-05 1.2668761e-02 3.6876744e-01 3.7720913e-01 2.0701981e-01
  3.4277882e-02]]
[[2.1563385e-04 3.4977317e-02 5.5354911e-01 2.9001933e-01 1.0511034e-01
  1.6128249e-02]]
[[1.8794394e-04 2.4457971e-02 3.9716104e-01 3.6549184e-01 1.8790442e-01
  2.4796769e-02]]
[[2.0665136e-04 3.0633226e-02 4.6130982e-01 3.3998719e-01 1.4685416e-01
  2.1008966e-02]]
[[1.2544464e-04 1.8121097e-02 3.6832556e-01 3.7189099e-01 2.1258932e-01
  2.8947607e-02]]


KeyboardInterrupt: 

In [57]:
image=os.getcwd()+'/test_images/kid_1.jpg'
image=cv.imread(image)
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
detected_faces = face_detector(gray, 1)
if(detected_faces):
    for i,face_rect in enumerate(detected_faces):
        shape=predictor(gray,face_rect)
        face_descriptor = facerec.compute_face_descriptor(frame,shape)
        face_descriptor=np.array(face_descriptor)
        face_descriptor=np.array(face_descriptor.reshape((-1,1*128)).astype(np.float32))
        print(model.predict(face_descriptor))

[[0.00087246 0.0554398  0.3229685  0.39640328 0.20473544 0.01958043]]


In [55]:
model_json = model.to_json()
with open("models/model_age_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model_age_1.h5")
print("Saved model to disk")

Saved model to disk


(194, 259, 3)